In [1]:
import pandas as pd
from docx import Document
import regex as re
import glob
from collections import Counter, defaultdict

In [2]:
fastaSequenceMirRE = re.compile(r'^[ATCGU]{15,30}$')
mirnaFasta = re.compile(r'>.*\n[ATCGU]{15,30}$')

In [32]:
print(fastaSequenceMirRE.search("GTGTGTTCTCTTATGTTGGTTG"))
print(fastaSequenceMirRE.search("Sequence: GTGTGTTCTCTTATGTTGGTTG"))

print(fastaSequenceMirRE.search("AUACAUGUGUGGCGUUGAUGGA"))


<regex.Match object; span=(0, 22), match='GTGTGTTCTCTTATGTTGGTTG'>
None
<regex.Match object; span=(0, 22), match='AUACAUGUGUGGCGUUGAUGGA'>


In [4]:
print(mirnaFasta.search("""
521_94_22_3p cand1

>seq_3430 22 bp
GAAAUACCAGUGGCUUACCGCA"""))

<regex.Match object; span=(21, 59), match='>seq_3430 22 bp\nGAAAUACCAGUGGCUUACCGCA'>


In [51]:
def is_empty(elem):
    if elem is None:
        return True

    if len(str(elem)) == 0:
        return True

    return False

def test_substrings_included(word, substrs):
    for x in substrs:
        if x in word:
            return True
    return False

def test_is_sequence( word):
    mirSequence = str(word).replace("Sequence: ", "").strip()
    match = fastaSequenceMirRE.search(mirSequence)

    if match:
        return True

    match = mirnaFasta.search(mirSequence)
    if match:
        return True

    return False
    
def column_contains_mirnacells(df, indexRow, xi):
    

    foundSeqs = 0
    for ri, row in df.loc[indexRow+1:min(10, df.shape[0]),].iterrows():
        
        cellContent = str(row[xi])

        #print('"'+cellContent+'"')
        #print(mirnaFasta.search(cellContent))

        if mirnaFasta.search(cellContent):
            foundSeqs += 1

    if foundSeqs > 0:
        return True

    print("[column_contains_mirnacells] Column", xi, "is not a valid miRNA Sequence", foundSeqs)
    return False

def column_contains_mirnas(df, indexRow, xi):

    foundSeqs = 0
    testedSeqs = 0
    for ri, row in df.loc[indexRow+1:min(10, df.shape[0]),].iterrows():
        
        word = str(row[xi])

        #print(word, test_is_sequence(word))
        if not is_empty(word):
            testedSeqs+= 1

        if test_is_sequence(word):
            foundSeqs += 1

    if testedSeqs>0 and foundSeqs/testedSeqs > 0.8:
        return True

    print("[column_contains_mirnas] Column", xi, "is not a valid miRNA Sequence", foundSeqs, testedSeqs)
    return False

def column_contains_highlighted_mirnas(table, indexRow, xi):

    foundSeqs = 0
    testedSeqs = 0
    for ri, row in enumerate(table.rows):
        
        if ri <= indexRow:
            continue

        for par in row.cells[xi].paragraphs:
            cellWasFilled = False
            mirnaWasFound = False

            for run in par.runs:

                specialText = bool(run.underline) or bool(run.bold) or bool(run.italic)

                if not is_empty(run.text):
                    cellWasFilled = True
                
                if specialText and test_is_sequence(run.text):
                    mirnaWasFound = True

            if cellWasFilled:
                testedSeqs += 1

            if mirnaWasFound: 
                foundSeqs += 1
        
    if  testedSeqs>0 and foundSeqs/testedSeqs > 0.8:
        return True

    print("[column_contains_highlighted_mirnas] Column", xi, "is not a valid miRNA Sequence", foundSeqs)
    return False

def getNameSeqColumns(df, table=None):

    indexRow = 0
    for i in range(0, min(10, df.shape[0])):
        idxCnt = Counter(["str" if not pd.isna(x) and not x == "nan" else "NAN" for x in df.loc[i,]])
        #print(i,idxCnt)
        if idxCnt.most_common(1)[0][0] == "str":
            indexRow = i
            break

    #print(indexRow)
    potentialColumns = list(df.loc[indexRow,])
    potentialColumns = [str(x).strip() for x in potentialColumns]
    
    mirName = None
    mirSeq = None
    mirReason = None    

    for xi, x in enumerate(potentialColumns):
        if test_substrings_included(x.upper(), ["NAME","ACCESSION", "MATURE MIRNA"]):
            mirName = xi

        elif test_substrings_included(x.upper(), ["MIRNA SEQUENCE","MATURE SEQUENCE", "SEQUENCE OF MATURE", "SEQUENCE"]) and column_contains_mirnas(df, indexRow, xi) and mirSeq is None:

            mirSeq = xi
            mirReason = "SEQ_COLUMN"
            
        elif test_substrings_included(x.upper(), ["MIRNA"]) and column_contains_mirnacells(df, indexRow, xi) and mirSeq is None:
            mirSeq = xi
            mirName = xi
            mirReason = "FASTA_COLUMN"

        elif not table is None and test_substrings_included(x.upper(), ["PRECURSOR SEQUENCE", "MIRNA SEQUENCE","MATURE SEQUENCE", "SEQUENCE OF MATURE", "SEQUENCE"]) and column_contains_highlighted_mirnas(table, indexRow, xi) and mirSeq is None:
            mirSeq = xi
            mirReason = "HIGHLIGHT_COLUMN"

    print("Name:", mirName, potentialColumns[mirName] if mirName != None else "")
    print("Seq: ", mirSeq, potentialColumns[mirSeq] if mirSeq != None else "")

    return indexRow, mirName, mirSeq, mirReason

In [33]:
test_is_sequence("AUACAUGUGUGGCGUUGAUGGA")

True

In [6]:
def extract_mirnas_from_document(infile):

    document = Document(infile)
    
    for table in document.tables:
        data = [[cell.text for cell in row.cells] for row in table.rows]
        df = pd.DataFrame(data)

        extract_mirnas_from_pandas_tabledf( df, table )



In [48]:
def extract_mirnas_from_pandas_tabledf( df, table=None ):
    startRow, mirName, mirSeq, reason = getNameSeqColumns(df, table)
    
    if mirName == None or mirSeq == None:
        print("Skipping")
        print(startRow, mirName, mirSeq)
        return

    print(mirName, mirSeq, reason)
    
    for ri, row in df.loc[startRow:,].iterrows():

        mirSeqText = str(row[mirSeq])
        mirNameText = str(row[mirName])

        if not is_empty(mirSeqText) and not is_empty(mirSeqText):

            mirSequence = mirSeqText.replace("Sequence: ", "")

            if test_is_sequence(mirSeqText):
                print(mirNameText, mirSeqText)

In [8]:
def extract_mirnas_from_pandas_table(infile):

    sheetsDF = pd.read_excel(infile, header=None, sheet_name=None)

    print("Sheets", [x for x in sheetsDF])

    for sheet in sheetsDF:

        df = sheetsDF[sheet]
        print(sheet, df.shape)
        if 0 in df.shape:
            print("Skipping sheet", sheet)
            continue

        extract_mirnas_from_pandas_tabledf( df )

In [9]:
def extract_mirnas_from_pandas_fasta(infile):

    sheetsDF = pd.read_excel(infile, header=None, sheet_name=None)

    fastaStartRE = re.compile(r'^>\S\S\S\S+')
    fastaSequenceMirRE = re.compile(r'^[ATCGU]{15,30}$')

    for sheet in sheetsDF:

        df = sheetsDF[sheet]

        for column in df.columns:

            bestRowSequence = None
            rowSequence = []
            
            for rowIdx, row in df.iterrows():

                fastaStart = fastaStartRE.search(str(row[column]))
                fastaSequenceMir = fastaSequenceMirRE.search(str(row[column]))

                if fastaStart:
                    #print(rowIdx, "fasta start")
                    rowSequence.append("start")
                if fastaSequenceMir:
                    #print(rowIdx, "fasta seq")
                    rowSequence.append("seq")

                if not fastaStart and not fastaSequenceMir:
                    if len(rowSequence) > 0 and (bestRowSequence == None or len(bestRowSequence) > len(rowSequence)):
                        bestRowSequence = rowSequence

                if rowIdx > 20:
                    break
            if len(rowSequence) > 0 and (bestRowSequence == None or len(bestRowSequence) > len(rowSequence)):
                bestRowSequence = rowSequence

            if not bestRowSequence is None:

                count = 0
                bestCount = 0
                for i in range(1, len(bestRowSequence)):

                    if bestRowSequence[i-1] != bestRowSequence[i]:
                        count += 1

                    else:
                        if count > bestCount:
                            bestCount = count

                if count > bestCount:
                            bestCount = count

                if bestCount > 0:
                    print(sheet, column, bestRowSequence)
                    print(sheet, column, bestCount)
                    print(df[column])

In [18]:
allFiles = []
allFiles += glob.glob("*/*.xlsx")
allFiles += glob.glob("*/*.XLSX")
allFiles += glob.glob("*/*.docx")
allFiles += glob.glob("*/*.DOCX")
allFiles = sorted(allFiles)
allFiles

['PMC7278893/PMC7278893_peerj-08-9369-s001.xlsx',
 'PMC7381279/PMC7381279_Data_Sheet_1.XLSX',
 'PMC7381279/PMC7381279_Data_Sheet_2.XLSX',
 'PMC7381279/PMC7381279_Data_Sheet_3.XLSX',
 'PMC7381279/PMC7381279_Data_Sheet_4.XLSX',
 'PMC7381279/PMC7381279_Data_Sheet_5.XLSX',
 'PMC7381279/PMC7381279_Data_Sheet_6.DOCX',
 'PMC7395633/PMC7395633_mmc2.docx',
 'PMC7773562/PMC7773562_mmc1.xlsx',
 'PMC8078050/PMC8078050_mmc1.docx',
 'PMC8078050/PMC8078050_mmc2.docx',
 'PMC8078050/PMC8078050_mmc3.docx',
 'PMC8078050/PMC8078050_mmc4.xlsx',
 'PMC8078050/PMC8078050_mmc5.xlsx',
 'PMC8078050/PMC8078050_mmc6.xlsx',
 'PMC8078050/PMC8078050_mmc7.xlsx',
 'PMC8078050/PMC8078050_mmc8.xlsx',
 'PMC8294073/PMC8294073_mmc1.docx',
 'PMC8294073/PMC8294073_mmc2.xlsx',
 'PMC8294073/PMC8294073_mmc3.xlsx',
 'PMC8294073/PMC8294073_mmc4.xlsx',
 'PMC8294073/PMC8294073_mmc5.xlsx',
 'PMC8294073/PMC8294073_mmc6.xlsx',
 'PMC8358877/PMC8358877_JCMM-25-7825-s001.docx',
 'PMC8358877/PMC8358877_JCMM-25-7825-s002.xlsx',
 'PMC8358877

In [52]:
for infile in ['PMC7381279/PMC7381279_Data_Sheet_3.XLSX']:
    print()
    print()
    print()
    print(infile)
    if infile.upper().endswith(".XLSX"):

        print("FASTA")
        extract_mirnas_from_pandas_fasta(infile)
        print("TABLE")
        extract_mirnas_from_pandas_table(infile)

    elif infile.upper().endswith(".DOCX"):
        extract_mirnas_from_document(infile)





PMC7381279/PMC7381279_Data_Sheet_3.XLSX
FASTA
Sheet1 0 ['start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start']
Sheet1 0 18
0      Supplementary file 3: List of predicted viral ...
1                                                    NaN
2                                         miRNA SARS-CoV
3                                      >5'stem-miRNA 106
4                                 CUACAACCAAGACAACGUUCAA
                             ...                        
338                                                  NaN
339                                                  NaN
340                                                  NaN
341                                                  NaN
342                                                  NaN
Name: 0, Length: 343, dtype: object
Sheet1 1 ['start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start', 'seq', 'start'

In [ ]:
df = pd.read_excel("PMC8078050/PMC8078050_mmc7.xlsx", header=None)


In [ ]:
df.loc[1,]

0                                     NaN
1                               Pre-miRNA
2                                     NaN
3                                    Rank
4                                    Name
5                             Orientation
6                                   Start
7                                    Size
8                                   Score
9                            End position
10                              Sequences
11                                    NaN
12                                    MFE
13                                      A
14                                      C
15                                      G
16                                      U
17                    Overall neucleotide
18                                  (G+C)
19                                 (G+C)%
20                                  (A+U)
21                                 (A+U)%
22                                   AMFE
23                                

In [ ]:
document = Document('PMC8358877/PMC8358877_JCMM-25-7825-s001.docx')

In [ ]:
for row in document.tables[0].rows:
    for run in row.cells[1].paragraphs[0].runs:

        specialText = bool(run.underline) or bool(run.bold) or bool(run.italic)
        
        if specialText and test_is_sequence(run.text):
            print(run.text)

UUGGCUACUAACAAUCUAGUUG
GAAAUACCAGUGGCUUACCGCA
GAAAUACCAGUGGCUUACCGCA
CGCGACGUGCUCGUACGUGGCU
GGCUACUAACAAUCUAGUUGUA
AAACUCAAACCCGUCCUUGAUU
UCUGCCUAUACAGUUGAACUCG
UUGUGGCAGAUGCUGUCAUAAA
UUGUGGCAGAUGCUGUCAUAAA
AAACAAUUGUUGAGGUUCAACC
CUUAUUACAGAGCAAGGGCUGG
CUUAUUACAGAGCAAGGGCUGG
UCUUAGCCUACUGUAAUAAGAC
UCUUAGCCUACUGUAAUAAGAC
GUGCACUUAUCUUAGCCUACUG
GUGCACUUAUCUUAGCCUACUG
GUGCACUUAUCUUAGCCUACUG
UGAGUUAGGUGAUGUUAGAGAA
AAGCAUCUAUGCCGACUACUAU
UUGAUAGUGUUACAGUGAAGAA
UUGAUAGUGUUACAGUGAAGAA
ACUUUGAUAAAGCUGGUCA
UGUAUCUAAAGUUGCGUAGUGA
CCCUAAUUAUGAAGAUUUACUC
UUGUUACAUGCACCAUAUGGAA
UUGGGUAGUGCUUUAUUAGAAG
AAGCUAAAAGACUGUGUUAUGU
GGUACAACAUUUACUUAUGCAU
UCGUGUUGUCUGUACUGCCGUU
CAGGGCUUUAACUGCAGAGUCA
CAGGGCUUUAACUGCAGAGUCACAUGUUGACACUGACUUAA
UCACAUGUUGACACUGACUUAA
UCACAUGUUGACACUGACUUAA
AGUCAUCGUCAACAACCUAGAC
UGAAAUGGUCAUGUGUGGCGGU
UGCUCGCAUAGUGUAUACAGCU
GCUUAAAGCACAUAAAGACAAA
UGCACAUGUAGCUAGUUGUGAU
UGAUCUUUAUAAGCUCAUGGGA
UCAACUGAAAUCUAUCAGGCCG
CGGCGGGCACGUAGUGUAGCUA
CUACACUAUGUCACUUGGUGCA
UUAUGCUUUGCUGUAUGA